In [10]:
"""
Zijing Gu
2017/12/19
extract time character from pictures in a folder and write to a csv file
"""


import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
import cv2
import numpy as np
import pytesseract
from PIL import Image
import os
import glob
import csv
import operator 
import re
import datetime

# Path of working folder on Disk
#src_path = "E:/Lab/Python/Project/OCR/"
#src_path = "/Users/JingJing/Desktop/time/lo/"


    
progtime=re.compile("^([1-9]|1[0-2]):[0-5][0-9]$")
    
    
    
#check if a character is integer
#3. Is the time sequential (greater) from the last time? -> if not delete
#4.Is the time sequential (greater) from the last time by more than one minute?
#5. If yes, is the time greater than the next time point? -> if yes delete
def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

#check if time format is correct
def is_time(time):
    if re.match(progtime, time):
        return True
    else:
        return False
#check AM and PM format
def is_apm(apm):
    apm = apm.upper()
    if apm == "AM" or apm == "PM":
        return True
    else:
        return False


def get_string(img_path, csvfile):
    # Read image with opencv
    img = cv2.imread(img_path)
    #print (img_path)
    
    #choose time area of the image
    cropped = img[235:275,1660:1840]
    # Convert to gray
    cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    
    cv2.imwrite("crop.png", cropped)
    im = Image.open("crop.png")
    pix = im.load()
    os.remove("crop.png")
    #convert any color that is not white to black, almost white to white
    for x in range(0,179): 
        for y in range(0,39):
            #print (pix[x,y])
            if pix[x,y] < 232:
                pix[x,y] = 0
            else:
                pix[x,y] = 255
     
    im.save("changed.png")
    
    #print pix[x,y] #Get the RGBA Value of the a pixel of an image
    #pix[x,y] = value # Set the RGBA Value of the image (tuple)
    #im.save("alive_parrot.png") # Save the modified pixels as png
    
    cropped  = cv2.imread("changed.png")
    os.remove("changed.png")
    
    kernel = np.ones((2,2),np.float32)/4
    cropped = cv2.filter2D(cropped,-1,kernel)
    
    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    cropped = cv2.dilate(cropped, kernel, iterations=1)
    cropped = cv2.erode(cropped, kernel, iterations=1)
    
    
    

    # Write image after removed noise
    #cv2.imwrite("removed_noise.png", img)

    #  Apply threshold to get image with only black and white
    #img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    # Write the image after apply opencv to do some ...
    cv2.imwrite("thres.png", cropped)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(Image.open("thres.png"))
    os.remove("thres.png")
    #os.remove("removed_noise.png")
    
    time =""
    
    apm = ""
    
    #extract digital time and AM/PM
    for ch in result:
        if (is_number(ch) or ch == ":"):
            time = time + ch
        elif (ch.upper() == "A" or ch.upper() == "P" or ch.upper() == "M") :
            apm = apm + ch.upper()
        

    outputSize = len(time) + len(apm)
    
        
    #Is it 6 or 7 characters long?  -> if not delete
    if outputSize == 6 or  outputSize == 7:
        #2. Are the first two digits between 1 and 12, is there a : , 
        #are the next 2 digits between 00 and 59, are the last two characters AM or PM 
        if is_time(time) and is_apm(apm):
            csvfile.write(img_path + ", " + time + " " + apm + "\n")
            #print (img_path + " " + time + apm)
 
    else:
        csvfile.write(img_path + ", " + "\n")
        #print (img_path)
   
    


print ('--- Start recognize text from images ---')

csvfile = open("data.csv", "w") 



for filename in glob.glob('*.png'):
    #print (filename + " " +get_string(filename))
    #csvfile.write(filename + ", " + get_string(filename) + "\n")
    
    get_string(filename, csvfile)
    

csvfile.close()

print ("------ Done -------")





#print("check time sequence")


--- Start recognize text from images ---
------ Done -------


In [11]:
import operator
import csv

sortcsv = open("sortedData.csv", "w") 
csv1 = open("data.csv", "r") 
#data = open("data.csv", "r")
csvsort = csv.reader(csv1, delimiter=",")
sort = sorted(csvsort,key = operator.itemgetter(0))
for row in sort:
    #print(row)
        
    sortcsv.write(row[0] + "," + row[1]+"\n")
    
csv1.close()
sortcsv.close()
print("done sorting")

done sorting


In [13]:
import re
progtime=re.compile("^([1-9]|1[0-2]):[0-5][0-9]$")

In [14]:
import pandas as pd
import numpy as np
import datetime

df = pd.read_csv('sortedData.csv', names=['photo number','time'])

df['hour'],df['combo'] = df['time'].str.split(':', 1).str
del df['time']

df['min'],df['AM/PM'] = df['combo'].str.split(' ', 1).str
del df['combo']

first = True

pre = datetime.time(0,0)
preM = ""

now = datetime.time(0,0)
nowM = ""

nex = datetime.time(0,0)
nexM = ""

exp = datetime.time(0,0)

for i in range(1, len(df.index) - 1):
    #only check on non empty cell
    if df.loc[i]['hour'] != " ":
        #set the first prev
        if first == True:
            pre = datetime.time(int(df.loc[i]['hour']),int(df.loc[i]['min']))
            preM = df.loc[i]['AM/PM']
            first = False
        
        else:
            now = datetime.time(int(df.loc[i]['hour']),int(df.loc[i]['min']))
            nowM = df.loc[i]['AM/PM']
            
            if pre.minute < 59:
                exp = datetime.time(pre.hour,pre.minute + 1)
            else :
                if pre.hour < 12:
                    exp = datetime.time(pre.hour + 1,0)
                else :
                    exp = datetime.time(1,0)
            
            #set next time
            for j in range(i+1, len(df.index) - 1):
                if df.loc[j]['hour'] != " ":
                    nex = datetime.time(int(df.loc[j]['hour']),int(df.loc[j]['min']))
                    nexM = df.loc[j]['AM/PM']
                    break
            
            #if now is different from its previous and next time range, delete
            if nowM != preM and nowM != nexM:
                df.loc[i]['hour'] = " "
                df.loc[i]['min'] = " "
                df.loc[i]['AM/PM'] = " "
            
            elif now < pre and nowM == preM and (not (pre.hour == 12 and now.hour != 12)):
                df.loc[i]['hour'] = " "
                df.loc[i]['min'] = " "
                df.loc[i]['AM/PM'] = " "
            
            #if same AM/PM but now is greater than next, delete
            elif now > exp and now > nex and (not (now.hour == 12 and nex.hour != 12)): 
                df.loc[i]['hour'] = " "
                df.loc[i]['min'] = " "
                df.loc[i]['AM/PM'] = " "
             
            else: 
                pre = now
                preM = nowM
                
    else:  
        df.loc[i]['hour'] = " "
        df.loc[i]['min'] = " "
        df.loc[i]['AM/PM'] = " "       
                       
df.to_csv("dataSortedModified.csv", sep='\t')     
print ('done')

done


In [ ]:

if re.match(progtime, "1:45"):
    print ("yes")
else:
    print ("no")

In [1]:
import datetime
a = datetime.time(0,30)
b = datetime.time(3,0)

print(a - b)

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [14]:
st = "12:45"
sts = st.split(":")
sts[1]

'45'

In [9]:
import datetime


a = datetime.time(9,20)
b = datetime.time(9,10)
print  ((a-b).total_seconds())

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [5]:
b = datetime.datetime.now()

In [7]:
print(a-b)

-1 day, 23:59:51.779082
